In [ ]:
# Import required libraries
import os
import time
import numpy as np
import pandas as pd
import polars as pl
from scipy.optimize import minimize, Bounds
from warnings import filterwarnings

# Import kaggle_evaluation only if available (competition environment)
try:
    import kaggle_evaluation.default_inference_server
except ModuleNotFoundError:
    print("Warning: kaggle_evaluation not available. This is expected when running locally.")
    kaggle_evaluation = None

filterwarnings("ignore")

## Competition Metric Implementation

The adjusted Sharpe ratio penalizes strategies that:
- Have volatility > 1.2x the market's volatility
- Underperform the market's mean excess return

In [ ]:
# Constants
MIN_INVESTMENT = 0.0
MAX_INVESTMENT = 2.0
TRADING_DAYS_PER_YEAR = 252


class ParticipantVisibleError(Exception):
    """Exception for competition submission errors"""
    pass


def score_metric(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    """
    Calculate the competition's adjusted Sharpe ratio metric.
    
    Args:
        solution: DataFrame with true values (forward_returns, risk_free_rate, etc.)
        submission: DataFrame with predictions
        row_id_column_name: Name of the row ID column (not used but required by API)
    
    Returns:
        Adjusted Sharpe ratio (capped at 1,000,000)
    """
    solut = solution.copy()
    solut['position'] = submission['prediction']

    # Validate position bounds
    if solut['position'].max() > MAX_INVESTMENT:
        raise ParticipantVisibleError(
            f'Position of {solut["position"].max()} exceeds maximum of {MAX_INVESTMENT}')
    if solut['position'].min() < MIN_INVESTMENT:
        raise ParticipantVisibleError(
            f'Position of {solut["position"].min()} below minimum of {MIN_INVESTMENT}')

    # Calculate strategy returns (weighted combination of risk-free rate and market returns)
    solut['strategy_returns'] = (
        solut['risk_free_rate'] * (1 - solut['position']) +
        solut['forward_returns'] * solut['position']
    )

    # Strategy metrics
    strategy_excess_returns = solut['strategy_returns'] - solut['risk_free_rate']
    strategy_excess_cumulative = (1 + strategy_excess_returns).prod()
    strategy_mean_excess_return = (strategy_excess_cumulative) ** (1 / len(solut)) - 1
    strategy_std = solut['strategy_returns'].std()

    if strategy_std == 0:
        raise ZeroDivisionError("Strategy standard deviation is zero")
    
    sharpe = strategy_mean_excess_return / strategy_std * np.sqrt(TRADING_DAYS_PER_YEAR)
    strategy_volatility = float(strategy_std * np.sqrt(TRADING_DAYS_PER_YEAR) * 100)

    # Market metrics
    market_excess_returns = solut['forward_returns'] - solut['risk_free_rate']
    market_excess_cumulative = (1 + market_excess_returns).prod()
    market_mean_excess_return = (market_excess_cumulative) ** (1 / len(solut)) - 1
    market_std = solut['forward_returns'].std()
    market_volatility = float(market_std * np.sqrt(TRADING_DAYS_PER_YEAR) * 100)

    # Volatility penalty (kicks in if strategy vol > 1.2x market vol)
    excess_vol = max(0, strategy_volatility / market_volatility - 1.2) if market_volatility > 0 else 0
    vol_penalty = 1 + excess_vol

    # Return penalty (kicks in if strategy underperforms market)
    return_gap = max(0, (market_mean_excess_return - strategy_mean_excess_return) * 100 * TRADING_DAYS_PER_YEAR)
    return_penalty = 1 + (return_gap ** 2) / 100

    # Adjusted Sharpe ratio
    adjusted_sharpe = sharpe / (vol_penalty * return_penalty)
    
    return min(float(adjusted_sharpe), 1_000_000)

## Optimize Strategy Parameters

Use Powell's method to find optimal daily positions that maximize the adjusted Sharpe ratio.

In [ ]:
start_time = time.time()

# Load training data
train_df = pd.read_csv("/kaggle/input/hull-tactical-market-prediction/train.csv", index_col="date_id")

# Focus on last 180 days for optimization (test period)
train_last_180 = train_df[-180:]


def objective_function(x):
    """
    Objective function to minimize (negative Sharpe ratio).
    
    Args:
        x: Array of daily positions (one per day)
    
    Returns:
        Negative adjusted Sharpe ratio (for minimization)
    """
    solution = train_last_180.copy()
    submission = pd.DataFrame({'prediction': x.clip(MIN_INVESTMENT, MAX_INVESTMENT)}, index=solution.index)
    return -score_metric(solution, submission, '')


# Initial guess: small positive position (0.05) for all days
x0 = np.full(180, 0.05)

# Run optimization
print("Starting optimization...")
result = minimize(
    objective_function,
    x0,
    method='Powell',
    bounds=Bounds(lb=MIN_INVESTMENT, ub=MAX_INVESTMENT),
    tol=1e-8
)

print(f"\nOptimization result:\n{result}")

# Store optimized predictions
optimized_predictions = result.x

# Create date_id to prediction mapping
date_ids = train_last_180.index.tolist()
predictions_dict = dict(zip(date_ids, optimized_predictions))

print(f"\nExecution time: {time.time() - start_time:.2f} seconds")
print(f"Optimized predictions for {len(predictions_dict)} dates")
print(f"Final score (adjusted Sharpe): {-result.fun:.6f}")

## Prediction Function

Main predict function used by the inference server.

In [ ]:
def predict(test: pl.DataFrame) -> float:
    """
    Generate prediction for a single test row.
    
    Args:
        test: Polars DataFrame with test data (single row)
    
    Returns:
        Predicted position (float between 0.0 and 2.0)
    """
    try:
        # Extract date_id from test data
        date_id = int(test.select("date_id").to_series().item())
        
        # Look up optimized prediction for this date
        pred = predictions_dict.get(date_id, None)
        
        if pred is None:
            # Fallback: neutral position if date not in optimized set
            print(f"Warning: date_id {date_id} not found in optimized predictions, using neutral position")
            return 1.0
        
        # Ensure prediction is within valid bounds
        return float(np.clip(pred, MIN_INVESTMENT, MAX_INVESTMENT))
        
    except Exception as e:
        print(f"Error in predict: {e}")
        # Ultimate fallback: neutral position
        return 1.0

## Inference Server

Set up the Kaggle evaluation server for submission.

In [ ]:
# Initialize inference server (only in Kaggle competition environment)
if kaggle_evaluation is not None:
    inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)
    
    # Run in competition mode or local testing mode
    if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
        inference_server.serve()
    else:
        inference_server.run_local_gateway(('/kaggle/input/hull-tactical-market-prediction/',))
else:
    print("Notebook execution complete. Predictions are ready for use.")